In [1]:
# see what your run definition looks like
with open(".azureml/explain.runconfig","r") as f:
    print(f.read())

{
  "script": "train.py",
  "arguments": [

  ],
  "target": "cpu",
  "framework": "Python",
  "communicator": "None",
  "autoPrepareEnvironment": true,
  "maxRunDurationSeconds": null,
  "nodeCount": 1,
  "environment": {
    "name": null,
    "version": null,
    "environmentVariables": {
      "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
      "userManagedDependencies": false,
      "interpreterPath": "python",
      "condaDependenciesFile": ".azureml/conda_dependencies.yml",
      "baseCondaEnvironment": null
    },
    "docker": {
      "enabled": true,
      "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
      "sharedVolumes": true,
      "gpuSupport": false,
      "shmSize": "1g",
      "arguments": [

      ],
      "baseImageRegistry": {
        "address": null,
        "username": null,
        "password": null
      }
    }
  },
  "history": {
    "outputCollection": true,
    "snapshotProject": true,
    "directoriesToWatch": [
      

In [3]:
# see what your package dependencies looks like
with open(".azureml/conda_dependencies.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - --index-url https://azuremlsdktestpypi.azureedge.net/AzureML-Contrib-Explain-Model-Gated/3010237
  - --extra-index-url https://pypi.python.org/simple
  - sklearn_pandas
  - azureml-defaults<0.1.50
  - azureml-contrib-explain-model<0.1.50
  - azureml-core<0.1.50
  - azureml-telemetry<0.1.50
  - azureml-explain-model<0.1.50
- scikit-learn



In [21]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn','pandas'], pip_packages=['sklearn-pandas','azureml-defaults'])

In [22]:
#submit and monitor a run
from azureml.core import RunConfiguration, Experiment, Run
from azureml.core.script_run_config import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute

ws = Workspace.from_config()
runconfig = RunConfiguration.load('.','explain')
src = ScriptRunConfig(source_directory='.',script='train_explain.py')

cpu_cluster = ComputeTarget(workspace=ws, name="cpucluster3")

src.run_config.target = cpu_cluster.name

src.run_config.environment = myenv


run = Experiment(ws,'generate-attrition-explainer').submit(config=src)

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [18]:
#!jupyter nbextension install  --py --user azureml.contrib.explain.model.visualize
#!jupyter nbextension enable  --py --user azureml.contrib.explain.model.visualize
#!source activate py36 && jupyter nbextension list

In [17]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard
from azureml.contrib.explain.model.explanation.explanation_client import ExplanationClient
from azureml.core.model import Model
from sklearn.externals import joblib

original_model = Model(ws, 'IBM_attrition_model')
model_path = original_model.download(exist_ok=True)

client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()
attrition_model = joblib.load(model_path)

x_test_path = './x_test.pkl'
run.download_file('x_test_ibm.pkl', output_file_path=x_test_path)

x_test = joblib.load(x_test_path)

ExplanationDashboard(global_explanation, attrition_model, x_test)

ExplanationWidget(value={'localExplanations': [[[0.011139991333791295, 0.0, 0.002154231393631467, 0.0, 0.00718…

In [30]:
#see what your score file looks like
with open("score.py","r") as f:
    print(f.read())

import pandas as pd
from sklearn.externals import joblib
from azureml.core.model import Model

def init():

    global original_model
    # global scoring_explainer

    # Retrieve the path to the model file using the model name
    # Assume original model is named original_prediction_model
    original_model_path = Model.get_model_path('IBM_attrition_model')
    # scoring_explainer_path = Model.get_model_path('IBM_attrition_explainer')

    original_model = joblib.load(original_model_path)
    # scoring_explainer = joblib.load(scoring_explainer_path)

def run(raw_data):
    # Get predictions and explanations for each data point
    data = pd.read_json(raw_data)
    # Make prediction
    predictions = original_model.predict(data)
    # Retrieve model explanations
    # local_importance_values = scoring_explainer.explain(data)
    # You can return any data type as long as it is JSON-serializable
    # return {'predictions': predictions.tolist(), 'local_importance_values': local_importan

In [33]:
with open("myenv.yml","r") as f:
    print(f.read())

name: project_environment
dependencies:
- python=3.6.2

- pip:
  - --extra-index-url https://pypi.python.org/simple
  - sklearn-pandas
  - azureml-defaults
  - azureml-core
  - azureml-telemetry
- scikit-learn
- pandas



In [34]:
#deploy
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig, Model

ws = Workspace.from_config()

# scoring_explainer_model = Model(ws, 'IBM_attrition_explainer')
attrition_model = Model(ws, 'IBM_attrition_model')

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "IBM_Attrition",  
                                                     "method" : "local_explanation"}, 
                                               description='Explain predictions on employee attrition')
                                               
inference_config = InferenceConfig(entry_script='score.py', 
                                   extra_docker_file_steps='Dockerfile', 
                                   runtime='python', 
                                   conda_file='myenv.yml')

# service = Model.deploy(ws, name='predictattritionsvc', models=[scoring_explainer_model, attrition_model], inference_config= inference_config, deployment_config=aciconfig)
service = Model.deploy(ws, name='predictattritionsvc1', models=[attrition_model], inference_config= inference_config, deployment_config=aciconfig)

Creating image
Running...........................................................
Succeeded
Image creation operation finished for image predictattritionsvc1:1, operation "Succeeded"


In [38]:
# get URI for querying
from azureml.core.webservice import AciWebservice
import requests
import json

# sample = '{"Age":{"899":49},"BusinessTravel":{"899":"Travel_Rarely"},"DailyRate":{"899":1098},"Department":{"899":"Research & Development"},"DistanceFromHome":{"899":4},"Education":{"899":2},"EducationField":{"899":"Medical"},"EnvironmentSatisfaction":{"899":4},"Gender":{"899":"Female"},"HourlyRate":{"899":21},"JobInvolvement":{"899":3},"JobLevel":{"899":2},"JobRole":{"899":"Laboratory Technician"},"JobSatisfaction":{"899":3},"MaritalStatus":{"899":"Single"},"MonthlyIncome":{"899":711},"MonthlyRate":{"899":2124},"NumCompaniesWorked":{"899":7},"OverTime":{"899":"Yes"},"PercentSalaryHike":{"899":8},"PerformanceRating":{"899":4},"RelationshipSatisfaction":{"899":3},"StockOptionLevel":{"899":0},"TotalWorkingYears":{"899":2},"TrainingTimesLastYear":{"899":0},"WorkLifeBalance":{"899":3},"YearsAtCompany":{"899":1},"YearsInCurrentRole":{"899":0},"YearsSinceLastPromotion":{"899":0},"YearsWithCurrManager":{"899":0}}'

sample = '{"Age":{"899":49},"BusinessTravel":{"899":"Travel_Rarely"},"DailyRate":{"899":1098},"Department":{"899":"Research & Development"},"DistanceFromHome":{"899":4},"Education":{"899":2},"EducationField":{"899":"Medical"},"EnvironmentSatisfaction":{"899":1},"Gender":{"899":"Male"},"HourlyRate":{"899":85},"JobInvolvement":{"899":2},"JobLevel":{"899":5},"JobRole":{"899":"Manager"},"JobSatisfaction":{"899":3},"MaritalStatus":{"899":"Married"},"MonthlyIncome":{"899":18711},"MonthlyRate":{"899":12124},"NumCompaniesWorked":{"899":2},"OverTime":{"899":"No"},"PercentSalaryHike":{"899":13},"PerformanceRating":{"899":3},"RelationshipSatisfaction":{"899":3},"StockOptionLevel":{"899":1},"TotalWorkingYears":{"899":23},"TrainingTimesLastYear":{"899":2},"WorkLifeBalance":{"899":4},"YearsAtCompany":{"899":1},"YearsInCurrentRole":{"899":0},"YearsSinceLastPromotion":{"899":0},"YearsWithCurrManager":{"899":0}}'

service = AciWebservice(ws,"predictattritionsvc1")
print(service.scoring_uri)

headers = {'Content-Type':'application/json'}

# send request to service
resp = requests.post(service.scoring_uri, sample, headers=headers)
print("prediction:", resp.text)
result = json.loads(resp.text)

#plot the feature importance for the prediction
# import numpy as np
# import matplotlib.pyplot as plt; plt.rcdefaults()
# import json

# labels = json.loads(sample)
# labels = labels.keys()
# objects = labels
# y_pos = np.arange(len(objects))
# performance = result["local_importance_values"][0][0]

# plt.bar(y_pos, performance, align='center', alpha=0.5)
# plt.xticks(y_pos, objects)
# locs, labels = plt.xticks()
# plt.setp(labels, rotation=90)
# plt.ylabel('Feature impact - leaving vs not leaving')
# plt.title('Local feature importance for prediction')

# plt.show()

http://ee542215-259e-4f4f-9b9b-0e946a53b492.eastus.azurecontainer.io/score
prediction: {"predictions": [0]}
